# Occupation. Parte 2. Agrupación!

### Step 1. Importamos las librerías

In [45]:
import pandas as pd
import sqlite3

### Step 2. Importar el conjunto de datos de esta [dirección](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user). 

## PANDAS


In [2]:
usuarios = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user", sep='|')

## SQL

In [46]:
#SQL
# Nos conectamos y creamos la db
conexion = sqlite3.connect('ocupaciones.db')

# Escribimos DataFrame en una SQLite tabla
usuarios.to_sql('ocupaciones', conexion, index=False, if_exists='replace')

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = conexion.cursor()

In [47]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

In [48]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0])

ocupaciones


### Step 3. Asignarlo a una variable llamada usuarios.

In [3]:
usuarios.head(10)

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [50]:
#SQL
query = '''
SELECT *
FROM ocupaciones
'''

df = sql_query(query)
df

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


### Step 4. Descubrir cuál es la edad media por ocupación

In [6]:
ocupacion_media = usuarios.groupby('occupation')[['age']].mean().sort_values(by=['age'], ascending=False)
ocupacion_media

,age
occupation,
retired,63.071429
doctor,43.571429
educator,42.010526
healthcare,41.562500
librarian,40.000000
administrator,38.746835
executive,38.718750
marketing,37.615385
lawyer,36.750000


In [54]:
#SQL
query = '''
SELECT occupation as ocupacion, AVG(age) as media_edad
FROM ocupaciones
GROUP BY occupation
ORDER BY media_edad DESC
'''

df = sql_query(query)
df

,ocupacion,media_edad
0,retired,63.071429
1,doctor,43.571429
2,educator,42.010526
3,healthcare,41.562500
4,librarian,40.000000
5,administrator,38.746835
6,executive,38.718750
7,marketing,37.615385
8,lawyer,36.750000
9,engineer,36.388060


### Step 5. Descubrir la proporción de hombres por ocupación y ordenarla de mayor a menor

In [30]:
usuarios_hombres = usuarios[usuarios['gender']=='M']
ocupaciones_hombres = usuarios_hombres['occupation'].value_counts()
ocupaciones_hombres

occupation
student          136
other             69
educator          69
engineer          65
programmer        60
administrator     43
executive         29
scientist         28
technician        26
writer            26
librarian         22
entertainment     16
marketing         16
artist            15
retired           13
lawyer            10
salesman           9
doctor             7
none               5
healthcare         5
homemaker          1
Name: count, dtype: int64

In [32]:
suma_ocupaciones = usuarios.occupation.value_counts()
suma_ocupaciones

occupation
student          196
other            105
educator          95
administrator     79
engineer          67
programmer        66
librarian         51
writer            45
executive         32
scientist         31
artist            28
technician        27
marketing         26
entertainment     18
healthcare        16
retired           14
lawyer            12
salesman          12
none               9
homemaker          7
doctor             7
Name: count, dtype: int64

In [33]:

proporcion_hombres = round((ocupaciones_hombres/suma_ocupaciones)*100, 2)
proporcion_hombres.sort_values(ascending=False)


occupation
doctor           100.00
engineer          97.01
technician        96.30
retired           92.86
programmer        90.91
executive         90.62
scientist         90.32
entertainment     88.89
lawyer            83.33
salesman          75.00
educator          72.63
student           69.39
other             65.71
marketing         61.54
writer            57.78
none              55.56
administrator     54.43
artist            53.57
librarian         43.14
healthcare        31.25
homemaker         14.29
Name: count, dtype: float64

In [94]:
#SQL --> Hay que hacer dos consultas y agruparlas en un select con la operacion correspondiente y un join
query = '''
WITH usuarios_hombres AS (
    SELECT occupation, COUNT(*) AS cantidad_hombres
    FROM ocupaciones
    WHERE gender = 'M'
    GROUP BY occupation
),
usuarios_totales AS (
    SELECT occupation, COUNT(*) AS suma_ocupaciones
    FROM ocupaciones
    GROUP BY occupation
)

SELECT
    usuarios_hombres.occupation AS ocupaciones,
    ROUND((usuarios_hombres.cantidad_hombres * 100.0) / usuarios_totales.suma_ocupaciones, 2) AS proporcion_hombres
FROM
    usuarios_hombres
JOIN
    usuarios_totales ON usuarios_hombres.occupation = usuarios_totales.occupation
ORDER BY
    proporcion_hombres DESC;


'''

df = sql_query(query)
df

,ocupaciones,proporcion_hombres
0,doctor,100.00
1,engineer,97.01
2,technician,96.30
3,retired,92.86
4,programmer,90.91
5,executive,90.63
6,scientist,90.32
7,entertainment,88.89
8,lawyer,83.33
9,salesman,75.00


### Step 6. Para cada ocupación, calcula las edades mínima y máxima

In [38]:
ocupacion_edad = usuarios.groupby('occupation')['age'].agg([('mean_age', 'mean'), ('min_age', 'min'), ('max_age', 'max')]).sort_values(by='mean_age', ascending=False)
ocupacion_edad



,mean_age,min_age,max_age
occupation,,,
retired,63.071429,51,73
doctor,43.571429,28,64
educator,42.010526,23,63
healthcare,41.562500,22,62
librarian,40.000000,23,69
administrator,38.746835,21,70
executive,38.718750,22,69
marketing,37.615385,24,55
lawyer,36.750000,21,53


In [55]:
#SQL
query = '''
SELECT occupation as ocupacion, AVG(age) as media_edad, 
MAX(age) as max_edad,
MIN(age) as min_edad
FROM ocupaciones
GROUP BY occupation
ORDER BY media_edad DESC
'''

df = sql_query(query)
df

,ocupacion,media_edad,max_edad,min_edad
0,retired,63.071429,73,51
1,doctor,43.571429,64,28
2,educator,42.010526,63,23
3,healthcare,41.562500,62,22
4,librarian,40.000000,69,23
5,administrator,38.746835,70,21
6,executive,38.718750,69,22
7,marketing,37.615385,55,24
8,lawyer,36.750000,53,21
9,engineer,36.388060,70,22


### Step 7. Para cada combinación de ocupación y género, calcular la edad promedio.

In [43]:
ocupacion_genero = usuarios.groupby(['occupation', 'gender'])[['age']].mean().sort_values(by=['occupation'], ascending=False)
ocupacion_genero

age
occupation    gender           
writer        M       35.346154
              F       37.631579
technician    M       32.961538
              F       38.000000
student       M       22.669118
              F       20.750000
scientist     M       36.321429
              F       28.333333
salesman      M       38.555556
              F       27.000000
retired       M       62.538462
              F       70.000000
programmer    M       33.216667
              F       32.166667
other         M       34.028986
              F       35.472222
none          M       18.600000
              F       36.500000
marketing     F       37.200000
              M       37.875000
librarian     M       40.000000
              F       40.000000
lawyer        M       36.200000
              F       39.500000
homemaker     M       23.000000
              F       34.166667
healthcare    M       45.400000
              F       39.818182
executive     F       44.000000
              M       38.172414
entertainment F       31.000000
              M       29.000000
engineer      M       36.600000
              F       29.500000
educator      M       43.101449
              F       39.115385
doctor        M       43.571429
artist        F       30.307692
              M       32.333333
administrator M       37.162791
              F       40.638889

In [58]:
#SQL
query = '''
SELECT occupation, gender, AVG(age) AS average_age
FROM ocupaciones
GROUP BY occupation, gender
ORDER BY occupation DESC;

'''

df = sql_query(query)
df

,occupation,gender,average_age
0,writer,F,37.631579
1,writer,M,35.346154
2,technician,F,38.000000
3,technician,M,32.961538
4,student,F,20.750000
5,student,M,22.669118
6,scientist,F,28.333333
7,scientist,M,36.321429
8,salesman,F,27.000000
9,salesman,M,38.555556
